This is the code that reproduces a Private Score of 0.94908.

- "Magic" is not used. The only token-saving method used is the AND omission shared at https://www.kaggle.com/competitions/uspto-explainable-ai/discussion/516104.
- Some refactoring has been done, but the core part remains unchanged.
- The solver includes a time-cutoff process, so the Private Score does not match exactly.

For more details: https://www.kaggle.com/competitions/uspto-explainable-ai/discussion/522202

## Setup

In [1]:
# import whoosh_utils
import os
import polars as pl

LOCAL = not os.path.exists("/kaggle")
DATADIR = "/kaggle/input/uspto-explainable-ai"
MY_DATADIR = "/kaggle/input/uspto2024dataset2"
MY_SCRIPTDIR = "/kaggle/input/uspto2024script2"
OUTDIR = "/kaggle/working"
TESTFILE = f"{DATADIR}/test.csv"
# TESTFILE = f"{MY_DATADIR}/test.csv"  # used for local validation
if LOCAL:
    DATADIR = "/workspaces/uspto/dataset"
    MY_DATADIR = "/workspaces/uspto/dataset/me"
    MY_SCRIPTDIR = "/workspaces/uspto/src"
    OUTDIR = "/workspaces/uspto/temporary"
    TESTFILE = f"{DATADIR}/test_2500/1.csv"

## Preprocess

In [2]:
test = pl.read_csv(TESTFILE)
display(test)

publication_number,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22,target_23,target_24,target_25,target_26,target_27,target_28,target_29,target_30,target_31,target_32,target_33,target_34,target_35,target_36,target_37,target_38,target_39,target_40,target_41,target_42,target_43,target_44,target_45,target_46,target_47,target_48,target_49
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""US-2017082634-…","""US-2020225242-…","""US-2010137151-…","""US-10607721-B2…","""US-6747273-B2""","""US-6940065-B2""","""US-7684934-B2""","""US-2019018928-…","""US-9040903-B2""","""US-7847245-B2""","""US-8501421-B2""","""US-9110076-B2""","""US-10617717-B2…","""US-8741556-B2""","""US-9312110-B2""","""US-7865312-B2""","""US-11428696-B2…","""US-11646185-B2…","""US-7603240-B2""","""US-9879304-B2""","""US-9354236-B2""","""US-2022365028-…","""US-2014051105-…","""US-2021239708-…","""US-10825672-B2…","""US-2022397578-…","""US-9746477-B2""","""US-2022221467-…","""US-2016139140-…","""US-2019353660-…","""US-9110074-B2""","""US-8604692-B2""","""US-2017168055-…","""US-7499807-B1""","""US-2003008327-…","""US-2003166007-…","""US-10598642-B2…","""US-10401337-B2…","""US-2005221500-…","""US-2017023547-…","""US-2003157575-…","""US-10197576-B2…","""US-11011359-B2…","""US-7020559-B1""","""US-8642349-B1""","""US-2009253156-…","""US-7783429-B2""","""US-2003060983-…","""US-10914740-B2…","""US-7858389-B2""","""US-9747412-B2"""
"""US-2017180470-…","""US-2017171349-…","""US-2017171301-…","""US-2017155712-…","""US-2017163642-…","""US-2017155740-…","""US-2017180760-…","""US-2017155927-…","""US-2014289307-…","""US-9550119-B2""","""US-2017163478-…","""US-9280384-B2""","""US-2017155928-…","""US-2017163479-…","""US-10798147-B2…","""US-2017171150-…","""US-11582325-B2…","""US-11057489-B2…","""US-2017150214-…","""US-10182126-B2…","""US-9456024-B1""","""US-2017155695-…","""US-2017163706-…","""US-2021234919-…","""US-2014073370-…","""US-10075387-B1…","""US-2012005274-…","""US-8180857-B2""","""US-2017171566-…","""US-11432171-B2…","""US-2018227220-…","""US-10231269-B2…","""US-2017163761-…","""US-2017180468-…","""US-2022263759-…","""US-11582153-B2…","""US-2019245912-…","""US-2021389960-…","""US-2017171147-…","""US-10958580-B2…","""US-10897506-B2…","""US-11586626-B1…","""US-10945130-B2…","""US-2023041645-…","""US-2014310385-…","""US-9749174-B1""","""US-2014344473-…","""US-2017142196-…","""US-9276860-B2""","""US-2017149934-…","""US-2017153909-…"
"""US-2018029544-…","""US-9597956-B2""","""US-2015197150-…","""US-2015034146-…","""US-2012132245-…","""US-2014137939-…","""US-2011073163-…","""US-10916749-B2…","""US-9350288-B2""","""US-2020105953-…","""US-8875828-B2""","""US-9016765-B1""","""US-2017104446-…","""US-2014174506-…","""US-5741370-A""","""US-2012240982-…","""US-2011226335-…","""US-2011146761-…","""US-2011315191-…","""US-2021193855-…","""US-11394071-B2…","""US-10211355-B2…","""US-2012080074-…","""US-5762720-A""","""US-2015194924-…","""US-2019319145-…","""US-2012315528-…","""US-2012103401-…","""US-2022367940-…","""US-8952236-B2""","""US-11431044-B2…","""US-2016043257-…","""US-6631943-B2""","""US-2013000689-…","""US-2011155221-…","""US-2023011595-…","""US-9722116-B2""","""US-2013118556-…","""US-2014345833-…","""US-2019035962-…","""US-9278617-B2""","""US-8748737-B2""","""US-4717790-A""","""US-2022320600-…","""US-10784390-B2…","""US-11152660-B2…","""US-2022407163-…","""US-8056664-B2""","""US-2022140428-…","""US-2011100421-…","""US-9242554-B2"""
"""US-2022408153-…","""US-7698238-B2""","""US-10593167-B2…","""US-2013268955-…","""US-2022337693-…","""US-11416128-B2…","""US-2023031160-…","""US-10834456-B2…","""US-8898687-B2""","""US-2020301398-…","""US-7610260-B2""","""US-2018

In [3]:
pubnum_list = []
query_pubnum_list = []
for row in test.iter_rows():
    query_pubnum_list.append(row[0])
    pubnum_list.extend(row[1:])
test_pubnums = pl.DataFrame({"publication_number": pubnum_list}).unique(maintain_order=True)
query_pubnums = pl.DataFrame({"publication_number": query_pubnum_list}).unique(maintain_order=True)
query_pubnums = query_pubnums.filter(~pl.col("publication_number").is_in(test_pubnums["publication_number"]))
print(test_pubnums.shape)
print(query_pubnums.shape)
display(test_pubnums)

(500, 1)
(10, 1)


publication_number
str
"""US-2020225242-…"
"""US-2010137151-…"
"""US-10607721-B2…"
"""US-6747273-B2"""
"""US-6940065-B2"""
…
"""US-864215-A"""
"""US-640142-A"""
"""US-647027-A"""


In [4]:
metadata = pl.scan_parquet(f"{DATADIR}/patent_metadata.parquet")
metadata = metadata.select(["publication_number","cpc_codes"]).collect()
# potential_negative = metadata.select(["publication_number"]).sample(1000, seed=42)
# potential_negative = metadata.select(["publication_number"]).sample(2000000, seed=42)
potential_negative = metadata.select(["publication_number"])
potential_negative = potential_negative.filter(~pl.col("publication_number").is_in(test_pubnums["publication_number"]))
potential_negative = potential_negative.filter(~pl.col("publication_number").is_in(query_pubnums["publication_number"]))
pubnums = pl.concat([test_pubnums.to_series(), query_pubnums.to_series(), potential_negative.to_series()])
n_test = test_pubnums.shape[0]
n_potential_negative = potential_negative.shape[0]
n_all = pubnums.shape[0]
print(n_test, n_potential_negative, n_all)
pubnums = pubnums.to_frame() # series -> dataframe
pubnums = pubnums.with_row_index("id")
print(pubnums.head())
pubnum2id = {}
for row in pubnums.rows():
    pubnum2id[row[1]] = row[0]
pubnums = pubnums.select(["publication_number"])
display(pubnums)

500 13307241 13307751
shape: (5, 2)
┌─────┬────────────────────┐
│ id  ┆ publication_number │
│ --- ┆ ---                │
│ u32 ┆ str                │
╞═════╪════════════════════╡
│ 0   ┆ US-2020225242-A1   │
│ 1   ┆ US-2010137151-A1   │
│ 2   ┆ US-10607721-B2     │
│ 3   ┆ US-6747273-B2      │
│ 4   ┆ US-6940065-B2      │
└─────┴────────────────────┘


publication_number
str
"""US-2020225242-…"
"""US-2010137151-…"
"""US-10607721-B2…"
"""US-6747273-B2"""
"""US-6940065-B2"""
…
"""US-T999003-I4"""
"""US-X1-I1"""
"""US-X7668-I1"""


In [5]:
test_pubnums_id = test_pubnums.with_row_index("id")
test_pubnum2id = {}
for row in test_pubnums_id.rows():
    test_pubnum2id[row[1]] = row[0]
test_pubnums_id = test_pubnums_id.select(["publication_number", "id"])
test_pubnums_id.write_csv(f"{OUTDIR}/vocab_pubnum.tsv", separator='\t', include_header=False)

In [6]:
test_id = test.select(
    [pl.col(col).map_elements(lambda x: pubnum2id.get(x, -1), return_dtype=pl.Int64).alias(col) for col in test.columns]
)
test_id.write_csv(f"{OUTDIR}/test.tsv", separator='\t', include_header=False)
display(test_id)

publication_number,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22,target_23,target_24,target_25,target_26,target_27,target_28,target_29,target_30,target_31,target_32,target_33,target_34,target_35,target_36,target_37,target_38,target_39,target_40,target_41,target_42,target_43,target_44,target_45,target_46,target_47,target_48,target_49
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
500,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
501,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
502,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
503,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
504,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249
505,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
506,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349
507,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
508,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449


In [7]:
nshot = pl.read_csv(f"{MY_DATADIR}/nshot.tsv", separator="\t")
test_nshot = test_pubnums.join(nshot, how="inner", on="publication_number")
print(len(test_nshot))
test_nshot.head()

def map_pubnum_to_id(pubnum):
    return pubnum2id.get(pubnum, None)

test_nshot = test_nshot.with_columns(
    pl.col("publication_number").map_elements(map_pubnum_to_id, return_dtype=pl.Int32).alias("pubid")
)
test_nshot = test_nshot.select(["pubid", "tokens"])
test_nshot.write_csv(f"{OUTDIR}/nshot.tsv", separator="\t", include_header=False)
display(test_nshot)

495


pubid,tokens
i32,str
392,"""detd:zzaleew"""
389,"""(detd:fomssr d…"
256,"""detd:gbtter"""
415,"""detd:m439085"""
248,"""(detd:wjjiw de…"
…,…
133,"""(cpc:H01L31/04…"
407,"""(cpc:G06V40/13…"
57,"""(ab:appointed …"


In [8]:
def create_and_write_vocab(df, key, write_key=None):
    if write_key is None:
        write_key = key
    words = set()
    for row in df.select(key).iter_rows():
        for word in row[0]:
            words.add(word)
    words = list(words)
    words.sort()
    words = pl.DataFrame({key: words})
    words = words.with_row_index("id")
    words = words.select([key, "id"])
    print(len(words))
    words.write_csv(f"{OUTDIR}/vocab_{write_key}.tsv", separator='\t', include_header=False)
    word2id = {}
    for row in words.rows():
        word2id[row[0]] = row[1]
    return word2id

def write_x2y(filepath: str, x2y):
    with open(filepath, "w") as f:
        f.write(f"{len(x2y)}\n")
        for y in x2y:
            f.write(f"{len(y)}")
            if len(y) == 0:
                f.write("\n")
                continue
            ystr = " ".join(map(str, y))
            f.write(f" {ystr}\n")

def calc_y2xs(df, id_x, id_y, vocab_x, vocab_y, write_key_y):
    y2xs = [[] for _ in range(len(vocab_y))]
    for row in df.iter_rows():
        x = row[id_x]
        xid = vocab_x[x]
        for y in row[id_y]:
            yid = vocab_y.get(y)
            if yid is None:
                continue
            y2xs[yid].append(xid)
    file = f"{OUTDIR}/{write_key_y}2pubnum.txt"
    write_x2y(file, y2xs)
    print(file)

def calc_x2ys(df, id_x, id_y, vocab_x, vocab_y, write_key_y):
    x2ys = [[] for _ in range(len(vocab_x))]
    for row in df.iter_rows():
        x = row[id_x]
        xid = vocab_x[x]
        x2ys[xid] = [vocab_y[y] for y in row[id_y]]
    file = f"{OUTDIR}/pubnum2{write_key_y}.txt"
    write_x2y(file, x2ys)  
    print(file)

In [9]:
df = pubnums.select("publication_number")
df = df.join(metadata, on="publication_number", how="left")
display(df)
df_test = df.join(test_pubnums, on="publication_number", how="inner")
cpc2id = create_and_write_vocab(df_test, "cpc_codes", "cpc")

ID_PUBNUM = 0
ID_CPC = 1

calc_x2ys(df_test, ID_PUBNUM, ID_CPC, test_pubnum2id, cpc2id, "cpc")
calc_y2xs(df, ID_PUBNUM, ID_CPC, pubnum2id, cpc2id, "cpc")

publication_number,cpc_codes
str,list[str]
"""US-2020225242-…","[""G01N2500/02"", ""G01N2560/00"", … ""H01J49/0036""]"
"""US-2010137151-…","[""G01N33/6818"", ""G16B15/00"", ""G16B30/00""]"
"""US-10607721-B2…","[""G01N2500/20"", ""G01N33/5008"", … ""G16B40/10""]"
"""US-6747273-B2""","[""C12Q1/48"", ""G01N30/7233"", … ""Y10T436/24""]"
"""US-6940065-B2""","[""G01N2030/8417"", ""G01N33/6803"", … ""H01J49/004""]"
…,…
"""US-T999003-I4""","[""G03C8/10""]"
"""US-X1-I1""","[""C01D3/08""]"
"""US-X7668-I1""",[]


1160
/kaggle/working/pubnum2cpc.txt
/kaggle/working/cpc2pubnum.txt


In [10]:
import gc
del metadata
del df, df_test
del cpc2id
del nshot, test_nshot
del potential_negative
gc.collect()

0

In [11]:
%%time
for field_info in [("description", "10k"), ("title", "400k"), ("abstract", "400k"), ("claims", "100k")]:
    field, freq = field_info
    df_x = pl.scan_parquet(f"{MY_DATADIR}/{field}_{freq}.parquet").collect()
    print(field, len(df_x))
    df_x_test = df_x.join(test_pubnums, on="publication_number", how="inner")
    print(field, len(df_x_test))
    display(df_x)
    x2id = create_and_write_vocab(df_x_test, field)
    ID_PUBNUM = 0
    ID_X = 1
    calc_x2ys(df_x_test, ID_PUBNUM, ID_X, test_pubnum2id, x2id, field)
    calc_y2xs(df_x, ID_PUBNUM, ID_X, pubnum2id, x2id, field)

description 13307658
description 500


publication_number,description
str,list[str]
"""US-X1-I1""","[""xqqqq""]"
"""US-X7668-I1""","[""7775mm"", ""broughtinto"", … ""jv""]"
"""US-X7982-I1""","[""galvanc"", ""hound"", … ""inuence""]"
"""US-X8736-I1""","[""harringtun""]"
"""US-107-A""",[]
…,…
"""US-T958613-I4""",[]
"""US-T959435-I4""",[]
"""US-T974866-I4""",[]


18110
/kaggle/working/pubnum2description.txt
/kaggle/working/description2pubnum.txt
title 13307658
title 500


publication_number,title
str,list[str]
"""US-X1-I1""","[""ashes"", ""ash"", … ""pearl""]"
"""US-X7668-I1""","[""curing"", ""applied"", … ""galvanic""]"
"""US-X7982-I1""","[""diseases"", ""electricity"", … ""certain""]"
"""US-X8736-I1""","[""galvanic""]"
"""US-107-A""","[""withdrawn""]"
…,…
"""US-T958613-I4""",[]
"""US-T959435-I4""",[]
"""US-T974866-I4""",[]


854
/kaggle/working/pubnum2title.txt
/kaggle/working/title2pubnum.txt
abstract 13307658
abstract 500


publication_number,abstract
str,list[str]
"""US-X1-I1""",[]
"""US-X7668-I1""",[]
"""US-X7982-I1""",[]
"""US-X8736-I1""",[]
"""US-107-A""",[]
…,…
"""US-T958613-I4""",[]
"""US-T959435-I4""",[]
"""US-T974866-I4""",[]


3230
/kaggle/working/pubnum2abstract.txt
/kaggle/working/abstract2pubnum.txt
claims 13307658
claims 500


publication_number,claims
str,list[str]
"""US-X1-I1""",[]
"""US-X7668-I1""",[]
"""US-X7982-I1""",[]
"""US-X8736-I1""",[]
"""US-107-A""",[]
…,…
"""US-T958613-I4""",[]
"""US-T959435-I4""",[]
"""US-T974866-I4""",[]


4596
/kaggle/working/pubnum2claims.txt
/kaggle/working/claims2pubnum.txt
CPU times: user 13min 3s, sys: 1min 21s, total: 14min 24s
Wall time: 11min 53s


In [12]:
import gc
del df_x, df_x_test
del pubnum2id, x2id
gc.collect()

0

## Solve and Validate

In [13]:
%%time
import subprocess

## Execution on 2500 lines takes approximately 4-6 hours.
result = subprocess.run(f"g++ -std=c++17 -O3 {MY_SCRIPTDIR}/main.cpp && ./a.out --field cpc --field title --field abstract --field claims --field description", shell=True, text=True, capture_output=True)

CPU times: user 25 ms, sys: 31.2 ms, total: 56.2 ms
Wall time: 3min 15s


## Submission

In [14]:
queries = result.stdout.strip().split("\n")
print(result.stderr)
assert len(queries) == len(test)
with open("submission.csv", "w") as f:
    f.write("publication_number,query\n")
    for i in range(len(queries)):
        pub = test["publication_number"][i]
        f.write(f"{pub},{queries[i]}\n")
!head -n 11 submission.csv

1188  {DATADIR} = {/kaggle/working/}
390  {testcases.size()} = {10}
391  {LAST_TEST_ID} = {509}
1211  {param_} = {max_depth=3,sampling_ratio=0.2,beam_width=100,timeout=0.2,solver=multi,active_fields=cpc|ti|ab|clm|detd|}
1213  {N_PUBNUM} = {13307750}
1131  {j, n_entity, n_token, positive, potential_negative, score, mean, timer.get()} = {0, 809, 50, 42, 0, 0.984869, 0.984869, 15.847}
1131  {j, n_entity, n_token, positive, potential_negative, score, mean, timer.get()} = {1, 277, 50, 36, 0, 0.953745, 0.969307, 9.796}
1131  {j, n_entity, n_token, positive, potential_negative, score, mean, timer.get()} = {2, 182, 50, 34, 4, 0.923651, 0.954088, 9.549}
1131  {j, n_entity, n_token, positive, potential_negative, score, mean, timer.get()} = {3, 324, 50, 36, 3, 0.942125, 0.951097, 9.758}
1131  {j, n_entity, n_token, positive, potential_negative, score, mean, timer.get()} = {4, 208, 49, 37, 0, 0.960238, 0.952926, 0.493}
1131  {j, n_entity, n_token, positive, potential_negative, score, mean, timer.g